<a href="https://colab.research.google.com/github/nmalby/APS/blob/rewrite/APS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#APS Explorer

In [2]:
import math
from functools import lru_cache

##Parameters

In [ ]:
'''
Parameters
  Configuration
    Shell
      Loader      - Belt, 1, 2, 3, 4, 6, 7, 8             - Multiple            CheckBox
      Modules
        Head      - AP, Heavy, AP, Thump, Chem, Special   - Multiple            Checkbox
        Fuse      - None, Single, Double                  - Single              Radio
        Base      - None, BB, Trace, Cavitate             - Single              Radio
    Gun
      Gauge       - 18-500                                - Single              Field/Slider
      Barrels     - 1-6                                   - Single              Radio
      Evacuator   - Y/N                                   - Single              Radio
      Ejector     - Y/N                                   - Single              Radio
      Rail        - Y/N                                   - Single              Radio
      Rail Limit  - 1000-20000                            - Single              Field/Slider

    Engine
      PPM         - 200-800                               - Single              Field/Slider
      PPV         - 10-200                                - Single              Field/Slider
      PPB         - 10-200                                - Single              Field/Slider

  Requirements
    AP            - 0-60                                  - Single              Field/Slider
    Velocity      - 500-2000                              - Multiple (min-max)  Slider
    (Pen Req)     -                                       - Single              Field

  Goal Weights
    KD            - 0-1                                   - Single              Field/Slider
    CD            - 0-1                                   - Single              Field/Slider
    ROF           - 0-1                                   - Single              Field/Slider
    (Uptime)      - 0-1 (Belt)                            - Single              Field/Slider
    Volume        - 0-1                                   - Single              Field/Slider
    Cost          - 0-1 (Build + 10min)                   - Single              Field/Slider
'''

In [4]:
constraints = {
    "Configuration": {
        "Shell": {
            "Modules": {
                "Projectile": {
                    "Head": ["AP", "Heavy", "Sabot", "Thump", "Chem", "Special"],
                    "Fuse": ["None", "Single", "Double"],
                    "Base": ["None", "BB", "Trace", "Supercav"]
                }
            }
        },
        "Gun": {
            "Loader": ["Belt", 1, 2, 3, 4, 6, 7, 8],
            "Gauge": (18, 500),
            "Barrels": (1, 6),
            "Evacuator": ["Y", "N"],
            "Ejector": ["Y", "N"],
            "Rail": ["Y", "N"],
            "RailLimit": (1000, 20000),
            "ClipPer": (0, 4),
            "BeltInput": (1, 6)
        },
        "Engine": {
            "PPM": (200, 800),
            "PPV": (10, 200),
            "PPB": (10, 200)
        },
        "Duration": (0, 60)
    },
    "Requirements": {
        "AP": (0, 100),
        "Velocity": {
            "min": 500,
            "max": 2000
        }
    },
    "GoalWeights": {
        "KD": (0, 1),
        "CD": (0, 1),
        "RoF": (0, 1),
        "Uptime": (0, 1),
        "Volume": (0, 1),
        "Cost": (0, 1)
    }
}

In [18]:
parameters = {
    "Configuration": {
        "Shell": {
            "Modules": {
                "Projectile": {
                    "Head": "AP",
                    "Body": {
                        "SolidBody": 0,
                        "ChemBody": 0,
                        "Fuse": 0,
                    },
                    "Base": None
                },
                "Casing": {
                    "GP": 10,
                    "RC": 0
                }
            }
        },
        "Gun": {
            "Loader": [4],
            "Gauge": {
                "min": 100,
                "max": 250
            },
            "Barrels": 1,
            "Evacuator": False,
            "Ejector": False,
            "Rail": False,
            "RailLimit": 1000,
            "ClipPer": 2,
            "BeltInput": 2
        },
        "Engine": {
            "PPM": 500,
            "PPV": 100,
            "PPB": 100
        },
        "Duration": 10
    },
    "Requirements": {
        "AP": 20,
        "Velocity": {
            "min": 500,
            "max": 2000
        },
        "PenReq": None
    },
    "GoalWeights": {
        "KD": 0.5,
        "CD": 0.5,
        "RoF": 0.5,
        "Uptime": 0.5,
        "Volume": 0.5,
        "Cost": 0.5
    }
}

##Implementation

In [122]:
class Shell:
    def __init__(self, configuration):
        """
        Initialize the Shell object with the given configuration.

        :param configuration: Dictionary containing shell, gun, and engine configurations.
        """

        # Shell Configuration
        self.modules = configuration["Configuration"]["Shell"]["Modules"] # Projectile, Casing
        self.projectile = configuration["Configuration"]["Shell"]["Modules"]["Projectile"]
        self.casing = configuration["Configuration"]["Shell"]["Modules"]["Casing"]

        # Gun Configuration
        self.loader = configuration["Configuration"]["Gun"]["Loader"]
        self.gauge = configuration["Configuration"]["Gun"]["Gauge"]
        self.barrels = configuration["Configuration"]["Gun"]["Barrels"]
        self.evacuator = configuration["Configuration"]["Gun"]["Evacuator"]
        self.ejector = configuration["Configuration"]["Gun"]["Ejector"]
        self.rail = configuration["Configuration"]["Gun"]["Rail"]
        self.rail_limit = configuration["Configuration"]["Gun"]["RailLimit"]
        self.rail_draw_scalar = configuration["Configuration"]["Gun"]["RailDrawScalar"]
        self.clip_per = configuration["Configuration"]["Gun"]["ClipPer"]
        self.belt_input = configuration["Configuration"]["Gun"]["BeltInput"]

        # Engine Configuration
        self.engine = configuration["Configuration"]["Engine"] # PPM, PPV, PPB, PPC

        # Misc
        self.duration = configuration["Configuration"]["Duration"]

    lur_cache_size = None

    #   KD

    #   CD

    #   RoF

    @lru_cache(maxsize=lur_cache_size)
    def reloadTime(self):
        reload = 17.5 * (self.gauge / 500)**1.35 * (2 + self.effectiveShellModuleCount())

        return reload if self.loader != "Belt" else reload * 1 # TODO: Calc

    def uptime(self):
        if self.loader != "Belt": return 1
        else:
            firingCycle = 1 # TODO: Calc
            loadingCycle = 1 # TODO: Calc
            return firingCycle/(loadingCycle + firingCycle)

    def cooldownTime(self):
        return max(32.8215 * (self.gauge / 500)**1.35 * self.casing["GP"]**0.35, 0)

    #   Volume

    def loaderVolume(self):
        ejectorVolume = 2 if self.ejector else 0
        return (self.loader + 1 + ejectorVolume) if self.loader != "Belt" else (2 + self.belt_input + ejectorVolume)

    def recoilVolume(self):
        return self.totalRecoil() / (self.reloadTime() * 120)

    def coolerVolume(self):
        if self.casing["GP"] == 0: return 0
        multiBarrelPenalty = 1 + (self.barrels - 1) * 0.2
        boreEvacuatorBonus = 0

        if self.evacuator: boreEvacuatorBonus = (0.15 / (0.35355 / math.sqrt(self.gauge / 1000))) * multiBarrelPenalty / self.barrels

        return ((((self.cooldownTime() * multiBarrelPenalty) / self.reloadTime() - boreEvacuatorBonus) /
            (1 + self.barrels * 0.05) / multiBarrelPenalty) * math.sqrt(self.gauge / 1000)) / 0.176775

    def chargerVolume(self):
        return self.railDraw() / self.reloadTime() / 200

    def magnetVolume(self):
        return self.railDraw() / 5000

    def engineVolume(self):
        return self.railDraw() / self.reloadTime() / self.engine["PPV"]

    def volumePerIntake(self):
        return max(
            self.loaderVolume() +
            self.recoilVolume() +
            self.coolerVolume() +
            self.chargerVolume() +
            self.magnetVolume() +
            self.engineVolume(),
            2
        )

    #   Cost

    def loaderCost(self):
        ejectorCost = costData["Ejector"] if self.ejector else 0
        if self.loader != "Belt": # compute for 1 intake, but fractionally distribute cost between autoloader and clip
            clipFraction = self.clip_per / (1 + self.clip_per)
            return ((1 - clipFraction) * (costData["Loader"][self.loader] + ejectorCost) +
                    clipFraction * costData["Clip"][self.loader] + costData["InputFeeder"])
        else:
            return costData["Loader"]["Belt"] + costData["Clip"][1] + self.belt_input * costData["InputFeeder"]

    def buildCost(self):
        return (
            self.loaderCost() +
            self.recoilVolume() * costData["RecoilAbsorber"] +
            self.coolerVolume() * costData["Cooler"] +
            self.chargerVolume() * costData["RailCharger"] +
            self.magnetVolume() * costData["RailMagnet"] +
            self.engineVolume() * self.engine["PPC"]
        )

    def projectileCost(self):
        return ((17.413 * self.effectiveShellModuleCount() * (self.gauge/1000)**1.8)
                + self.railDraw() / self.engine["PPM"])

    def costPerIntake(self):
        return self.buildCost() + (self.projectileCost() / self.reloadTime()) * self.duration * 60

    #   Misc

    def loaderLength(self):
        return 1000 if self.loader == "Belt" else (self.loader * 1000)

    def projectileLength(self):
        length = 0
        for kind, module in self.projectile.items():
            if module is not None:
                if isinstance(module, dict):  # Check if the module is a dictionary containing counts
                    for mod_name, count in module.items():
                        max_length = moduleData.get(kind, {}).get(mod_name, {}).get("MaxLength", 500)
                        length += min(self.gauge, max_length) * count  # Multiply by the count
                else:
                    max_length = moduleData.get(kind, {}).get(module, {}).get("MaxLength", 500)
                    length += min(self.gauge, max_length)
        return length

    def casingLength(self):
        return (self.casing["GP"] + (self.casing["RC"] or 0)) * self.gauge

    def totalLength(self): # will be used to apply penalty if totalLength > loaderLength
        return self.projectileLength() + self.casingLength()

    def effectiveProjectileModuleCount(self):
        return self.projectileLength() / self.gauge

    def effectiveShellModuleCount(self):
        return self.effectiveProjectileModuleCount() + (self.casing["GP"] + (self.casing["RC"] or 0)) / 4


    @lru_cache(maxsize=lur_cache_size)
    def railDraw(self):
        return (min(self.rail_limit, self.maxDraw()) * self.rail_draw_scalar) if self.rail else 0

    def maxDraw(self):
        return 12500 * (self.gauge**3/125000000)**0.65 * (self.effectiveProjectileModuleCount() + (0.5 * self.casing["RC"]))

    def totalRecoil(self):
        return (self.gauge**3/125000000)**0.65 * self.casing["GP"] * 2500 + self.railDraw()


###Test

In [ ]:
cnfg = {
    "Configuration": {
        "Shell": {
            "Modules": {
                "Projectile": {
                    "Head": "AP",
                    "Body": {
                        "SolidBody": 2,
                        "ChemBody": 2,
                        "Fuse": 1,
                    },
                    "Base": None
                },
                "Casing": {
                    "GP": 10,
                    "RC": 0,
                }
            }
        },
        "Gun": {
            "Loader": 8,
            "Gauge": 500,
            "Barrels": 1,
            "Evacuator": False,
            "Ejector": False,
            "Rail": True,
            "RailLimit": 30000,
            "RailDrawScalar": 1,
            "ClipPer": 2,
            "BeltInput": 2
        },
        "Engine": {
            "PPM": 500,
            "PPV": 100,
            "PPB": 100,
            "PPC": 0.1,
        },
        "Duration": 10
    },
}

testShell = Shell(cnfg)
print(testShell.totalRecoil())

##Data

In [30]:
loaderLengths = ["Belt", 1, 2, 3, 4, 6, 8]

moduleData = {
    "Head": {
        "AP": {
            "VelocityMod": 1.6,
            "KineticDamageMod": 1.0,
            "ArmorPierceMod": 1.65,
            "ChemMod": 1.0,
            "ShortName": "AP",
        },
        "Sabot": {
            "VelocityMod": 1.6,
            "KineticDamageMod": 0.85,
            "ArmorPierceMod": 2.5,
            "ChemMod": 0.25,
            "ShortName": "Sab",
        },
        "Heavy": {
            "VelocityMod": 1.45,
            "KineticDamageMod": 1.65,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "ShortName": "Hvy",
        },
        "HP": {
            "VelocityMod": 1.45,
            "KineticDamageMod": 1.2,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "ShortName": "HP",
        },
        "Chem": {
            "VelocityMod": 1.3,
            "KineticDamageMod": 1.0,
            "ArmorPierceMod": 0.1,
            "ChemMod": 1.0,
            "IsChem": True,
            "ShortName": "Chem",
        },
        "Special": {
            "VelocityMod": 1.45,
            "KineticDamageMod": 0.1,
            "ArmorPierceMod": 0.1,
            "ChemMod": 1.0,
            "IsChem": True,
            "ShortName": "Spc",
        }
    },
    "Body": {
        "SolidBody": {
            "VelocityMod": 1.1,
            "KineticDamageMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "ShortName": "Sol",
        },
        "SabotBody": {
            "VelocityMod": 1.1,
            "KineticDamageMod": 0.8,
            "ArmorPierceMod": 1.4,
            "ChemMod": 0.25,
            "ShortName": "Sab",
        },
        "ChemBody": {
            "VelocityMod": 1.0,
            "KineticDamageMod": 1.0,
            "ArmorPierceMod": 0.1,
            "ChemMod": 1.0,
            "ShortName": "Chem",
        },
        "Fuse": {
            "VelocityMod": 1.0,
            "KineticDamageMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "MaxLength": 100,
            "ShortName": "Fuse",
        },
        "UpperBound": {  # Highest values from Solid/Sabot/Chem
            "VelocityMod": 1.1,
            "KineticDamageMod": 1.0,
            "ArmorPierceMod": 1.4,
            "ChemMod": 1.0,
            "ShortName": "UpperBound",
        },
        "LowerBound": {  # Lowest values from Solid/Sabot/Chem
            "VelocityMod": 1.0,
            "KineticDamageMod": 0.8,
            "ArmorPierceMod": 0.1,
            "ChemMod": 1.0,
            "ShortName": "LowerBound",
        },
    },
    "Base": {
        "BaseBleeder": {
            "VelocityMod": 1.0,  # special +0.15 bonus handled elsewhere
            "KineticDamageMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "MaxLength": 100,
            "ShortName": "BB",
        },
        "Tracer": {
            "VelocityMod": 1.0,
            "KineticDamageMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "MaxLength": 100,
            "ShortName": "Tracer",
        },
        "Supercav": {
            "VelocityMod": 1.0,
            "KineticDamageMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 0.75,
            "MaxLength": 100,
            "ShortName": "Supercav",
        },
    }
}


In [8]:
costData = {
    "InputFeeder": 50,
    "Cooler": 50,
    "RecoilAbsorber": 80, # 80 per meter, longer ones scale linearly
    "RailCharger": 400,
    "RailMagnet": 300,
    "Ejector": 10,
    "Loader": {
        "Belt": 600,
        1: 240,
        2: 300,
        3: 330,
        4: 360,
        5: 390,
        6: 420,
        7: 450,
        8: 480,
    },
    "Clip": {
        "Belt": 160,
        1: 160,
        2: 200,
        3: 220,
        4: 240,
        5: 260,
        6: 280,
        7: 300,
        8: 320,
    }
}